In [2]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.metrics import roc_auc_score, accuracy_score
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.cuda.amp as amp
import uuid


In [3]:

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Custom Dataset for Multi-Label Chest X-rays
class ChestXrayDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        # Explicitly define the 15 disease columns based on the CSV
        self.label_cols = [
            'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion',
            'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass',
            'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax'
        ]
        # Verify label columns exist in CSV
        missing_cols = [col for col in self.label_cols if col not in self.data.columns]
        if missing_cols:
            raise ValueError(f"Missing columns in CSV: {missing_cols}")
        self.num_classes = len(self.label_cols)
        print(f"Dataset loaded with {self.num_classes} disease classes: {self.label_cols}")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.data.iloc[idx]['FullPath'])
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            raise
        image = np.array(image)
        
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        try:
            labels = self.data.iloc[idx][self.label_cols].values.astype(np.float32)
        except Exception as e:
            print(f"Error processing labels for index {idx}: {e}")
            print(f"Row data: {self.data.iloc[idx]}")
            raise
        return image, torch.tensor(labels)

# Focal Loss for multi-label classification
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

# Data Augmentation and Preprocessing
train_transform = A.Compose([
    A.Resize(224, 224),
    A.RandomCrop(200, 200, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

# Load Data
csv_file = '/media/sahil/Windows1/projectshit/balanced_data/archive (8)/new_labels_with_path.csv'
img_dir = '/media/sahil/Windows1/projectshit/balanced_data/archive (8)/new_images/new_images'
try:
    dataset = ChestXrayDataset(csv_file, img_dir, transform=None)
except Exception as e:
    print(f"Error loading dataset: {e}")
    raise

# Split Dataset
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

# Model Setup with Debugging
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}, Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

try:
    model = models.densenet121(weights='IMAGENET1K_V1')
    print("DenseNet-121 loaded successfully with pre-trained weights.")
except Exception as e:
    print(f"Error loading DenseNet-121: {e}")
    raise

# Verify model parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {num_params}")
if num_params == 0:
    raise ValueError("Model has no trainable parameters. Check model initialization.")

num_classes = dataset.num_classes  # Should be 15
model.classifier = nn.Linear(model.classifier.in_features, num_classes)
model = model.to(device)
print(f"Model moved to {device} with classifier modified for {num_classes} classes.")

# Verify parameters again after modification
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters after classifier modification: {num_params}")
if num_params == 0:
    raise ValueError("Model has no trainable parameters after classifier modification.")

# Check parameters before optimizer
params = list(model.parameters())
if len(params) == 0:
    raise ValueError("Model parameters are empty before optimizer initialization.")
print(f"Number of parameter tensors: {len(params)}")

# Loss and Optimizer
criterion = FocalLoss(gamma=2.0, alpha=0.25)
try:
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    print("Adam optimizer initialized successfully.")
except Exception as e:
    print(f"Error initializing Adam optimizer: {e}")
    raise

scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
scaler = amp.GradScaler()

# Training Loop
num_epochs =10
best_auc = 0.0
model_path = f'chest_xray_densenet_{uuid.uuid4()}.pth'

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        with amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item() * images.size(0)
        if i % 10 == 0:
            print(f"Epoch {epoch+1}, Batch {i}, Loss: {loss.item():.4f}")
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    
    # Validation
    model.eval()
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_preds.append(torch.sigmoid(outputs).cpu().numpy())
            val_labels.append(labels.cpu().numpy())
    
    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    
    # Compute Metrics
    auc_scores = [roc_auc_score(val_labels[:, i], val_preds[:, i]) for i in range(num_classes)]
    accuracies = [accuracy_score(val_labels[:, i], (val_preds[:, i] > 0.5).astype(int)) for i in range(num_classes)]
    
    mean_auc = np.mean(auc_scores)
    mean_accuracy = np.mean(accuracies)
    print(f'Validation Mean AUC: {mean_auc:.4f}, Mean Accuracy: {mean_accuracy:.4f}')
    for i, (auc, acc) in enumerate(zip(auc_scores, accuracies)):
        print(f'Disease {dataset.label_cols[i]}: AUC={auc:.4f}, Accuracy={acc:.4f}')
    
    # Save Best Model
    if mean_auc > best_auc:
        best_auc = mean_auc
        torch.save(model.state_dict(), model_path)
        print(f'Saved best model with AUC: {best_auc:.4f}')
    
    scheduler.step()

# Load Best Model for Final Evaluation
model.load_state_dict(torch.load(model_path))
model.eval()
final_preds = []
final_labels = []
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        final_preds.append(torch.sigmoid(outputs).cpu().numpy())
        final_labels.append(labels.cpu().numpy())

final_preds = np.concatenate(final_preds)
final_labels = np.concatenate(final_labels)

# Final Metrics
final_accuracies = [accuracy_score(final_labels[:, i], (final_preds[:, i] > 0.5).astype(int)) for i in range(num_classes)]
for i, acc in enumerate(final_accuracies):
    print(f'Final Accuracy for {dataset.label_cols[i]}: {acc:.4f}')

# Check Accuracy Threshold
below_threshold = [(dataset.label_cols[i], acc) for i, acc in enumerate(final_accuracies) if acc < 0.85]
if below_threshold:
    print('Diseases below 85% accuracy:', below_threshold)
    print('Consider fine-tuning: increase epochs, adjust learning rate, or apply TTA.')
else:
    print('All diseases achieved 85%+ accuracy!')

Dataset loaded with 15 disease classes: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax']
Using device: cuda
GPU: NVIDIA GeForce GTX 1650, Memory: 4.09 GB
DenseNet-121 loaded successfully with pre-trained weights.
Total trainable parameters: 7978856
Model moved to cuda with classifier modified for 15 classes.
Total trainable parameters after classifier modification: 6969231
Number of parameter tensors: 364
Adam optimizer initialized successfully.
Epoch 1, Batch 0, Loss: 0.0592
Epoch 1, Batch 10, Loss: 0.0302
Epoch 1, Batch 20, Loss: 0.0229
Epoch 1, Batch 30, Loss: 0.0183
Epoch 1, Batch 40, Loss: 0.0195
Epoch 1, Batch 50, Loss: 0.0208
Epoch 1, Batch 60, Loss: 0.0199
Epoch 1, Batch 70, Loss: 0.0223
Epoch 1, Batch 80, Loss: 0.0248
Epoch 1, Batch 90, Loss: 0.0192
Epoch 1, Batch 100, Loss: 0.0210
Epoch 1, Batch 110, Loss: 0.0207
Epoch 1, B